In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import os
import pickle
from typing import List, Dict, Any, Optional

d:\Genai.labs assignment\assignment\Ecommerce_Assistant_Challenge 2025\genailabs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# product_service/vector_store.py
# Import necessary libraries here...

class ProductVectorStore:
    """
    Vector store for product information retrieval.
    Uses embeddings to perform semantic search on product data.
    """
    
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        """Initialize the vector store with a model for embeddings."""
        # Initialize model, index, and other necessary components
        self.model_name = model_name
        self.model = SentenceTransformer(self.model_name)
        self.embeddings = None
        self.product_ids = None
    
    def load_data(self, file_path):
        """Load product data from CSV file."""
        
        try: 
            df = pd.read_csv(file_path)
            self.product_df = df

            print(f"Loaded {len(df)} products from {file_path}")

            return self.preprocess_data(df)
        
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def preprocess_data(self, df=None):
        """Preprocess product data for embedding generation."""
        if df is None: 
            df = self.product_df

        # Create a copy to avoid modifying the original
        processed_df = df.copy()
        
        # Generate product IDs if they don't exist
        if "product_id" not in processed_df.columns:
            processed_df["product_id"] = processed_df.index

        # Check for duplicate titles and remove duplicates
        duplicate_count = processed_df.duplicated(subset = ['title'], keep = 'first').sum()
        if duplicate_count > 0:
            print(f"Found {duplicate_count} duplicate products - keeping only first occurrences")
            processed_df = processed_df.drop_duplicates(subset = ['title'], keep = 'first')
        
        # Fill missing values in text fields
        text_columns = ['title', 'description', 'features', 'categories', 'details']
        for col in text_columns:
            if col in processed_df.columns:
                processed_df[col] = processed_df[col].fillna('')
        
        # Normalize text - lowercase, remove extra whitespace
        for col in text_columns:
            if col in processed_df.columns:
                processed_df[col] = processed_df[col].str.lower().str.strip()
        
        # Store processed dataframe
        self.product_df = processed_df
        self.product_ids = processed_df['product_id'].tolist()

        return processed_df

    
    def create_embeddings(self):
        """Generate embeddings for product data."""

        # Ensure we have data to embed
        if self.product_df is None or len(self.product_df) == 0:
            print("No data available for embedding generation")
            return None
        
        # Get all text columns 
        text_columns = ['main_category', 'title', 'features', 'description', 'categories', 'details']
        
        # Create text representations by combining all fields
        print("Creating text representations for embedding...")
        product_texts = []
        
        for _, row in self.product_df.iterrows():
            # Combine all text fields into one string
            text_parts = []
            for col in text_columns:
                if col in row and not pd.isna(row[col]) and row[col]:
                    text_parts.append(f"{col}: {row[col]}")
            
            # Add numeric information
            if 'average_rating' in row and not pd.isna(row['average_rating']):
                text_parts.append(f"rating: {row['average_rating']}")
            
            if 'price' in row and not pd.isna(row['price']):
                text_parts.append(f"price: {row['price']}")
    
            if ('imputed_columns' in row and row['imputed_columns'] != None and row['imputed_columns']!= []):
                text_parts.append(f"imputed_columns: {row['imputed_columns']}")
            
            # Combine all parts with spaces
            product_text = " ".join(text_parts)
            product_texts.append(product_text)
        
        # Generate embeddings
        print(f"Generating embeddings for {len(product_texts)} products...")
        try:
            embeddings = self.model.encode(product_texts)
            self.embeddings = embeddings
            
            print(f"Successfully created embeddings of shape {embeddings.shape}")
            return embeddings
        
        except Exception as e:
            print(f"Error generating embeddings: {e}")
            return None
    
    def build_index(self):
        """Build search index from embeddings."""
        # Check if embeddings exist
        if self.embeddings is None:
            print("No embeddings available. Call create_embeddings() first.")
            return None
        
        try:
            # Get embedding dimensions
            vector_dimension = self.embeddings.shape[1]
            
            # Create a new index with the correct dimensions
            self.index = faiss.IndexFlatL2(vector_dimension)
            
            # Add the embeddings to the index
            self.index.add(self.embeddings)
            
            print(f"Successfully built index with {self.index.ntotal} vectors")
            return self.index
        
        except Exception as e:
            print(f"Error building index: {e}")
            return None
    
    def save_index(self, index_path, data_path):
        """
        Save the FAISS index and product data to disk.

        Args:
            index_path: Path to save the FAISS index
            data_path: Path to save the product data

        Returns:
            bool: True if saving was successful, False otherwise
        """
        # Check if index exists
        if self.index is None:
            print("No index available to save. Call build_index() first.")
            return False

        try:
            # Create directories if they don't exist
            os.makedirs(os.path.dirname(index_path), exist_ok=True)
            os.makedirs(os.path.dirname(data_path), exist_ok=True)

            # Save the FAISS index
            faiss.write_index(self.index, index_path)

            # Save the embeddings
            if self.embeddings is not None:
                embeddings_path = os.path.splitext(index_path)[0] + "_embeddings.npy"
                np.save(embeddings_path, self.embeddings)
                print(f"Embeddings saved to {embeddings_path}")

            # Save the product dataframe
            self.product_df.to_pickle(data_path)

            print(f"Index saved to {index_path}")
            print(f"Product data saved to {data_path}")
            return True

        except Exception as e:
            print(f"Error saving index: {e}")
            return False
    
    def load_index(self, index_path, data_path):
        """
        Load a FAISS index and product data from disk.

        Args:
            index_path: Path to the saved FAISS index
            data_path: Path to the saved product data

        Returns:
            bool: True if loading was successful, False otherwise
        """
        try:
            # Load the FAISS index
            self.index = faiss.read_index(index_path)

            # Load the embeddings if available
            embeddings_path = os.path.splitext(index_path)[0] + "_embeddings.npy"
            if os.path.exists(embeddings_path):
                self.embeddings = np.load(embeddings_path)
                print(f"Loaded embeddings of shape {self.embeddings.shape}")

            # Load the product dataframe
            self.product_df = pd.read_pickle(data_path)

            # Recreate product_ids
            if 'product_id' in self.product_df.columns:
                self.product_ids = self.product_df['product_id'].tolist()
            else:
                self.product_ids = list(range(len(self.product_df)))

            print(f"Loaded index with {self.index.ntotal} vectors")
            print(f"Loaded {len(self.product_df)} products")
            return True

        except Exception as e:
            print(f"Error loading index: {e}")
            return False
    
    def search(self, query, top_k=5):
        """
        Search for products similar to the query.

        Args:
            query: Text query to search for
            top_k: Number of results to return

        Returns:
            List of dictionaries containing product information
        """
        # Check if index exists
        if self.index is None:
            print("No index available. Call build_index() first.")
            return []

        try:
            # Convert query to embedding
            query_embedding = self.model.encode([query])

            # Normalize the query embedding for cosine similarity
            faiss.normalize_L2(query_embedding)

            # Search the index
            distances, indices = self.index.search(query_embedding, top_k)

            # Fetch the actual product data
            results = []
            for i, idx in enumerate(indices[0]):
                if idx < len(self.product_df):
                    # Get the product data
                    product = self.product_df.iloc[idx].to_dict()

                    # Add distance score (lower is better for L2 distance)
                    product['search_score'] = float(distances[0][i])

                    # Add to results
                    results.append(product)

            print(f"Found {len(results)} products matching the query: '{query}'")
            return results

        except Exception as e:
            print(f"Error searching index: {e}")
            return []
    
    def search_by_category(self, query, category, top_k=5):
        """
        Search for products within a specific category.
        
        Args:
            query: Text query to search for
            category: Category to filter by
            top_k: Number of results to return
        
        Returns:
            List of dictionaries containing product information
        """
        # First get more results than we need
        results = self.search(query, top_k=top_k*3)
        
        # Filter by category
        filtered_results = []
        for product in results:
            if 'main_category' in product and product['main_category'] == category:
                filtered_results.append(product)

        
        # Return the top k results
        return filtered_results[:top_k]

    def get_product_by_id(self, product_id):
        """
        Retrieve product details by ID.

        Args:
            product_id: ID of the product to retrieve

        Returns:
            Dictionary containing product information, or None if not found
        """
        # Check if we have product data
        if self.product_df is None: 
            print("No product data available.")
            return None

        try:
            # Find the product by ID
            if 'product_id' in self.product_df.columns:
                product = self.product_df[self.product_df['product_id'] == product_id]

                # If product found, return it as a dictionary
                if not product.empty:
                    return product.iloc[0].to_dict()

            # If we reach here, the product wasn't found
            print(f"Product with ID {product_id} not found.")
            return None

        except Exception as e:
            print(f"Error retrieving product: {e}")
            return None

In [13]:
# test_vector_store.py
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import time

# Import the ProductVectorStore class
# from vector_store import ProductVectorStore

def test_init():
    """Test initialization of the vector store."""
    print("\n===== Testing Initialization =====")
    vector_store = ProductVectorStore()
    print(f"Model name: {vector_store.model_name}")
    print(f"Model loaded: {vector_store.model is not None}")
    assert vector_store.model is not None, "Model should be initialized"
    return vector_store

def test_load_data(vector_store):
    """Test loading data from CSV."""
    print("\n===== Testing load_data() =====")
    # Define the path to your CSV file
    data_path = "D:/Genai.labs assignment/assignment\Ecommerce_Assistant_Challenge 2025/new_data/Product_Information_Dataset.csv"
    
    # Ensure file exists
    assert os.path.exists(data_path), f"Data file not found at {data_path}"
    
    # Load the data
    start_time = time.time()
    df = vector_store.load_data(data_path)
    load_time = time.time() - start_time
    
    # Check if data loaded successfully
    assert df is not None, "DataFrame should not be None"
    assert len(df) > 0, "DataFrame should not be empty"
    
    print(f"Loaded {len(df)} products in {load_time:.2f} seconds")
    print(f"DataFrame columns: {df.columns.tolist()}")
    
    # Check if product_id was created
    assert 'product_id' in df.columns, "product_id column should exist"
    
    return df

def test_preprocess_data(vector_store):
    """Test preprocessing of product data."""
    print("\n===== Testing preprocess_data() =====")
    
    # Get the raw dataframe size
    raw_df_size = len(vector_store.product_df)
    
    # Process the data
    start_time = time.time()
    processed_df = vector_store.preprocess_data()
    process_time = time.time() - start_time
    
    # Check if processing worked
    assert processed_df is not None, "Processed DataFrame should not be None"
    
    # Check for duplicates removal
    if raw_df_size > len(processed_df):
        print(f"Removed {raw_df_size - len(processed_df)} duplicates")
    
    # Check text normalization (sample a few rows)
    if len(processed_df) > 0:
        sample_row = processed_df.iloc[0]
        if 'title' in sample_row:
            title = sample_row['title']
            assert title.lower() == title, "Title should be lowercase"
            assert title == title.strip(), "Title should be stripped of whitespace"
    
    print(f"Preprocessed {len(processed_df)} products in {process_time:.2f} seconds")
    return processed_df

def test_create_embeddings(vector_store):
    """Test creation of embeddings."""
    print("\n===== Testing create_embeddings() =====")
    
    # Create embeddings
    start_time = time.time()
    embeddings = vector_store.create_embeddings()
    embed_time = time.time() - start_time
    
    # Check if embeddings were created
    assert embeddings is not None, "Embeddings should not be None"
    assert vector_store.embeddings is not None, "Embeddings should be stored in vector_store"
    
    # Check shape of embeddings
    num_products = len(vector_store.product_df)
    embed_dim = vector_store.model.get_sentence_embedding_dimension()
    assert embeddings.shape == (num_products, embed_dim), f"Embeddings shape should be ({num_products}, {embed_dim})"
    
    print(f"Created embeddings of shape {embeddings.shape} in {embed_time:.2f} seconds")
    return embeddings

def test_build_index(vector_store):
    """Test building the FAISS index."""
    print("\n===== Testing build_index() =====")
    
    # Build index
    start_time = time.time()
    index = vector_store.build_index()
    build_time = time.time() - start_time
    
    # Check if index was built
    assert index is not None, "Index should not be None"
    assert hasattr(index, 'ntotal'), "Index should be a FAISS index"
    assert index.ntotal == len(vector_store.embeddings), "Index should contain all embeddings"
    
    print(f"Built index with {index.ntotal} vectors in {build_time:.2f} seconds")
    return index

def test_save_and_load_index(vector_store):
    """Test saving and loading the index."""
    print("\n===== Testing save_index() and load_index() =====")
    
    # Create directory if it doesn't exist
    os.makedirs("services/product_service/test_index", exist_ok=True)
    
    # Define paths
    index_path = "services/product_service/test_index/test_product_index.bin"
    data_path = "services/product_service/test_index/test_product_data.pkl"
    
    # Save index
    start_time = time.time()
    save_result = vector_store.save_index(index_path, data_path)
    save_time = time.time() - start_time
    
    # Check if save was successful
    assert save_result, "Save operation should return True"
    assert os.path.exists(index_path), f"Index file should exist at {index_path}"
    assert os.path.exists(data_path), f"Data file should exist at {data_path}"
    
    print(f"Saved index and data in {save_time:.2f} seconds")
    
    # Create a new vector store
    new_vector_store = ProductVectorStore()
    
    # Load index
    start_time = time.time()
    load_result = new_vector_store.load_index(index_path, data_path)
    load_time = time.time() - start_time
    
    # Check if load was successful
    assert load_result, "Load operation should return True"
    assert new_vector_store.index is not None, "Index should not be None after loading"
    assert new_vector_store.product_df is not None, "Product DataFrame should not be None after loading"
    assert len(new_vector_store.product_df) == len(vector_store.product_df), "Loaded DataFrame should have same size"
    
    print(f"Loaded index and data in {load_time:.2f} seconds")
    return new_vector_store

def test_search(vector_store):
    """Test search functionality."""
    print("\n===== Testing search() =====")
    
    # Define test queries
    test_queries = [
        "acoustic guitar",
        "high quality microphone",
        "music equipment with good ratings"
    ]
    
    for query in test_queries:
        # Search for products
        start_time = time.time()
        results = vector_store.search(query, top_k=3)
        search_time = time.time() - start_time
        
        # Check if search returned results
        assert isinstance(results, list), "Search results should be a list"
        assert len(results) <= 3, "Search should return at most top_k results"
        
        print(f"\nQuery: '{query}'")
        print(f"Found {len(results)} results in {search_time:.4f} seconds:")
        
        # Display results
        for i, result in enumerate(results):
            print(f"{i+1}. {result.get('title', 'No title')} (Score: {result.get('search_score', 0):.4f})")

def test_search_by_category(vector_store):
    """Test search by category functionality."""
    print("\n===== Testing search_by_category() =====")
    
    # Get a sample category from the data
    if 'main_category' in vector_store.product_df.columns:
        sample_category = vector_store.product_df['main_category'].iloc[0]
        
        # Search by category
        query = "high quality"
        start_time = time.time()
        results = vector_store.search_by_category(query, sample_category, top_k=3)
        search_time = time.time() - start_time
        
        # Check if search returned results
        assert isinstance(results, list), "Search results should be a list"
        
        print(f"Query: '{query}' in category '{sample_category}'")
        print(f"Found {len(results)} results in {search_time:.4f} seconds:")
        
        # Display results
        for i, result in enumerate(results):
            if len(results) > 0:
                print(f"{i+1}. {result.get('title', 'No title')} (Score: {result.get('search_score', 0):.4f})")
    else:
        print("Skipping category search test - no main_category column in data")

def test_get_product_by_id(vector_store):
    """Test retrieving a product by ID."""
    print("\n===== Testing get_product_by_id() =====")
    
    # Get a sample product ID
    if len(vector_store.product_ids) > 0:
        sample_id = vector_store.product_ids[0]
        
        # Get product by ID
        start_time = time.time()
        product = vector_store.get_product_by_id(sample_id)
        retrieval_time = time.time() - start_time
        
        # Check if product was retrieved
        assert product is not None, f"Product with ID {sample_id} should be found"
        assert isinstance(product, dict), "Retrieved product should be a dictionary"
        
        print(f"Retrieved product with ID {sample_id} in {retrieval_time:.4f} seconds:")
        print(f"Title: {product.get('title', 'No title')}")
        print(f"Price: ${product.get('price', 0):.2f}")
        print(f"Rating: {product.get('average_rating', 0):.1f}")
    else:
        print("Skipping product retrieval test - no product IDs available")

def run_all_tests():
    """Run all tests in sequence."""
    print("===== Running all ProductVectorStore tests =====")
    
    # Test initialization
    vector_store = test_init()
    
    # Test loading data
    test_load_data(vector_store)
    
    # Test preprocessing
    test_preprocess_data(vector_store)
    
    # Test embedding creation
    test_create_embeddings(vector_store)
    
    # Test index building
    test_build_index(vector_store)
    
    # Test saving and loading
    loaded_vector_store = test_save_and_load_index(vector_store)
    
    # Test search functionality
    test_search(loaded_vector_store)
    
    # Test category search
    test_search_by_category(loaded_vector_store)
    
    # Test product retrieval
    test_get_product_by_id(loaded_vector_store)
    
    print("\n===== All tests completed successfully! =====")

if __name__ == "__main__":
    run_all_tests()

<>:25: SyntaxWarning: invalid escape sequence '\E'
<>:25: SyntaxWarning: invalid escape sequence '\E'
C:\Users\goldr\AppData\Local\Temp\ipykernel_24032\4250044334.py:25: SyntaxWarning: invalid escape sequence '\E'
  data_path = "D:/Genai.labs assignment/assignment\Ecommerce_Assistant_Challenge 2025/new_data/Product_Information_Dataset.csv"


===== Running all ProductVectorStore tests =====

===== Testing Initialization =====
Model name: all-MiniLM-L6-v2
Model loaded: True

===== Testing load_data() =====
Loaded 5000 products from D:/Genai.labs assignment/assignment\Ecommerce_Assistant_Challenge 2025/new_data/Product_Information_Dataset.csv
Found 169 duplicate products - keeping only first occurrences
Loaded 4831 products in 0.60 seconds
DataFrame columns: ['main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'store', 'categories', 'details', 'parent_asin', 'imputed_columns', 'product_id']

===== Testing preprocess_data() =====
Found 1 duplicate products - keeping only first occurrences
Removed 1 duplicates
Preprocessed 4830 products in 0.06 seconds

===== Testing create_embeddings() =====
Creating text representations for embedding...
Generating embeddings for 4830 products...
Successfully created embeddings of shape (4830, 384)
Created embeddings of shape (4830, 384) in 246.58 s

In [11]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.
